<a href="https://colab.research.google.com/github/mathrhino/pytorch/blob/main/torch_mnist_total.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import os 
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [39]:
seed = 1

batch_size= 64
test_batch_size =64

no_cuda=False

In [40]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

#Preprocess#

In [41]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                   transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,),(0.3081,))
                   ])),
                   batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, transform=transforms.Compose([
                                                                         transforms.ToTensor(),
                                                                         transforms.Normalize((0.1307,),(0.3081))
    ])),
    batch_size=test_batch_size, shuffle=True
) 

In [42]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1=nn.Conv2d(1,20,5,1)
    self.conv2=nn.Conv2d(20,50,5,1)
    self.fc1=nn.Linear(4*4*50,500)
    self.fc2=nn.Linear(500, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x,2,2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x,2,2,)
    
    x=x.view(-1,4*4*50)
    x=F.relu(self.fc1(x))
    x=self.fc2(x)
    return F.log_softmax(x, dim=1)

In [43]:
model = Net().to(device)

In [44]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)


In [45]:
params = list(model.parameters())
for i in range(8):
  print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


In [46]:
model.train()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [47]:
data, target = next(iter(train_loader))

In [48]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [49]:
data, target = data.to(device), target.to(device)

In [50]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [51]:
#optimizer의 gradient를 clear해서 새로운 최적화 값 찾기

optimizer.zero_grad()

In [52]:
output = model(data)

In [53]:
#nll = negative loss likelihood
loss = F.nll_loss(output, target)

In [54]:
#back propagation -> gradient 구하기
loss.backward()

In [55]:
#parameter update 필요
optimizer.step()

In [56]:
epochs = 1 
log_interval =100

In [57]:
for epoch in range(1, epochs+1):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()

      #학습이 잘 되는 지 파악하기
    if batch_idx % log_interval ==0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100* batch_idx/len(train_loader), loss.item()))
  model.eval()

  test_loss =0
  correct =0

  with torch.no_grad():
    for data, target in test_loader:
      data, target= data.to(device), target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)

  print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'. format(test_loss, correct, len(test_loader.dataset), 100.* correct/len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.295366
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.220284
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.136019
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.953842
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.498457
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.254111
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.842578
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.706257
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.586765
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.612415


In [58]:
#Evaluation

test_loss =0 
correct =0

with torch.no_grad():
  data, target = next(iter(test_loader))
  data, target = data.to(device), target.to(device)
  output = model(data)

  test_loss += F.nll_loss(output, target, reduction='sum').item()

  pred = output.argmax(dim=1, keepdim=True)
  correct = pred.eq(target.view_as(pred)).sum().item()

In [60]:
output.shape

torch.Size([64, 10])

In [61]:
pred.shape

torch.Size([64, 1])

In [62]:
target.view_as(pred).shape

torch.Size([64, 1])

In [63]:
pred.eq(target.view_as(pred)).sum().item()/ 64

0.765625

In [64]:
test_loss

41.98169708251953

In [65]:
correct

49

In [67]:
test_loss /= len(test_loader.dataset)

In [68]:
test_loss 

0.0041981697082519535

In [70]:
model.eval()

test_loss =0
correct =0

with torch.no_grad():
  for data, target in test_loader:
    data, target= data.to(device), target.to(device)
    output = model(data)
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'. format(test_loss, correct, len(test_loader.dataset), 100.* correct/len(test_loader.dataset)))


Test set: Average Loss: 0.4869, Accuracy: 8609/10000 (86%)

